# Kernel Ridge Regression

## Introduction

KRR is a one of the ML methods to perform regression (fitting).

In practice, we use kernel function $d$ that measures the distance between two data points.
With the function we build the kernel matrix:

\begin{equation}
K_{ij} =  d(M_{i}, M_{j}) - \alpha I_{ij}
\end{equation}

for all pairs of samples $M_i$ and $M_j$ in the training set. $I_{ij}$ is the identity matrix and $\alpha$ is a hyperparameter chosen between 0 and 1.

When we want to predict the output of a new sample or set of samples that were not in the training set, we need to first evaluate the distance between them and the training samples:

\begin{equation}
D_{ij} =  d(T_{i}, M_{j})
\end{equation}

where $T_i$ is one of the new samples. The output prediction $\mathbf{y_T}$ for the test set $T$ is obtained by:

\begin{equation}
\mathbf{y_T} = \mathbf{y_M} \cdot K^{-1} \cdot D^T
\end{equation}

where $\mathbf{y_M}$ is the vector of known outputs for the training set $M$.

## Tutorial 1: fitting simple toy data

We are going to use KRR to fit the same data from the neural network notebook.

In [ ]:
from pyKRR import KRRsolver  # import our KRR solver object
import numpy, random, math
from matplotlib import pyplot as plt

# This function takes a 2-element vector p=[x,y] and computes z
# same as in the NN example!
def Model(p):
    result = [math.sin(0.4*p[0] - 0.3*p[1] + 0.2*p[0]*p[1])]
    result[0] += (2*random.random()-1)*0.1;
    return result

# generate some data to play with...
# training points
nTrainPts = 100
trainIn = 2*numpy.random.rand(nTrainPts,2) -1 # (x,y) points beween -1 and 1

# validation points
nValidPts = 40
validIn = 2*numpy.random.rand(nValidPts,2)-1
 
# apply Model to the points to compute z for the two sets
trainOut = numpy.apply_along_axis(Model, 1, trainIn).flatten()
validOut = numpy.apply_along_axis(Model, 1, validIn).flatten()

# create a new solver
solver = KRRsolver()

# set the hyperparameter
solver.alpha = 0.5

# call its training function with the training inputs and outputs
solver.Train(trainIn, trainOut)

# get a list of predicted outputs for the validation inputs
predict = solver.Evaluate(validIn)

# do the regression plot
plt.plot(validOut, predict, 'o')
plt.plot([-1,1], [-1,1], '-')
plt.xlabel('correct output')
plt.ylabel('KRR output')
plt.show()

## Chemical Data

We will now use KRR to model the atomisation energy of organic molecules.

All we need is a <i>distance</i> (or kernel) function that compares two molecules. Finding such function is non-trivial since it has to compare object of different size, regardless of their relative rotation and position.

We introduce Coulomb matrixes (CMs) as compressed descriptors. Given atomic positions and charges, the Coulomb matrix C is:<br><br>
C<sub>ij</sub> = Z<sub>i</sub> Z<sub>j</sub> / R<sub>ij</sub> &nbsp;&nbsp;&nbsp; when i != j<br>
C<sub>ii</sub> = Z<sub>i</sub><sup>2.4</sup>/2<br>

The CM is then diagonalised and the sorted eigenvalues are used as descriptor, which is invariant w.r.t. the ordering of the atoms, and rotation-translations of the system.

The kernel function <i>d</i> can be defined as the euclidean norm between the eigenvalues of CMs. If a molecule has more atoms than the other, we just complement the eigenvalues with zeros (already present in the loaded database).

More info in: M. Rupp, et al., <br>
Fast and Accurate Modeling of Molecular Atomization Energies with Machine Learning,<br>
Physical Review Letters, <b>108</b>, 058301 (2012)

## Exercise: KRR for energy prediction

We will now apply KRR to the prediction of atomisation energy of small organic molecules.
The database consists of 1000 molecules with up to 9 heavy atoms from the GDB-9 set.


In [ ]:
# The database has 1000 small molecules, max 23 atoms

# load the pre-processed Coulomb matrixes
molecules = numpy.loadtxt('data/krr-evals.txt')

# load the DFT atomisation energy of each molecule
energies = numpy.loadtxt('data/krr-PBE0energies.txt')

# split the data into training/validation
nTrain = 700
# take some as a training set
trainM = molecules[0:nTrain]
trainE = energies[0:nTrain]

nValid = 300
# take some other as validation
validM = molecules[nTrain:nTrain+nValid]
validE = energies[nTrain:nTrain+nValid]

In [ ]:
# create a new KRR solver

# train it

# evaluate it with the validation set

#make a regression plot

#comment on the results